In [1]:
import keras 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D
from keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from keras.utils.vis_utils import plot_model
from keras.layers import Input, GlobalAveragePooling2D
from keras import models
from keras.models import Model
import tensorflow

In [2]:
from keras.preprocessing.image import ImageDataGenerator
train_data_dir = 'train'
validation_data_dir = 'validation'
img_rows = 48
img_cols = 48
batch_size = 32
num_classes = 5

train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=30,
                shear_range=0.3,
                zoom_range=0.3,
                width_shift_range=0.4,
                height_shift_range=0.4,
                horizontal_flip=True,
                fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                train_data_dir,
                color_mode='grayscale',
                target_size=(48,48),
                batch_size=batch_size,
                class_mode='categorical',
                shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                        validation_data_dir,
                        color_mode='grayscale',
                        target_size=(img_rows,img_cols),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=True)



Found 24282 images belonging to 5 classes.
Found 5937 images belonging to 5 classes.


In [3]:
shape_x = 48
shape_y = 48
nClasses = 5

In [4]:
input_img = Input(shape=(shape_x, shape_y, 1))

### 1st layer
layer_1 = Conv2D(256, (1,1), padding='same', activation='relu')(input_img)
layer_1 = Conv2D(256, (3,3), padding='same', activation='relu')(layer_1)
layer_1 = BatchNormalization()(layer_1)
layer_1 = MaxPooling2D(pool_size=(2,2))(layer_1)
layer_1 = Dropout(0.2)(layer_1)

layer_2 = Conv2D(128, (3,3), padding='same', activation='relu')(input_img)
layer_2 = BatchNormalization()(layer_2)
layer_2 = MaxPooling2D(pool_size=(2,2))(layer_2)
layer_2 = Dropout(0.2)(layer_2)

layer_3 = Conv2D(64, (3,3), padding='same', activation='relu')(input_img)
layer_3 = Conv2D(32, (3,3), padding='same', activation='relu')(layer_3)
layer_3 = BatchNormalization()(layer_3)
layer_3 = MaxPooling2D(pool_size=(2,2))(layer_3)
layer_3 = Dropout(0.2)(layer_3)

mid_1 = tensorflow.keras.layers.concatenate([layer_1, layer_2, layer_3], axis = 3)

In [5]:

flat_1 = Flatten()(mid_1)
dense_1 = Dense(64,kernel_initializer = 'he_normal',activation='relu')(flat_1)
dense_1 = BatchNormalization()(dense_1)
dense_1 = Dropout(0.5)(dense_1)
output = Dense(nClasses, activation='softmax')(dense_1)

In [6]:
model = Model([input_img], output)

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('inception_model.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

callbacks = [checkpoint]


In [8]:
history=model.fit_generator(
                train_generator,
                epochs=25,
                callbacks=callbacks,
                validation_data=validation_generator)

C:\Users\20190\AppData\Local\Temp/ipykernel_19032/1311612697.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(


Epoch 1/25
758/759 [============================>.] - ETA: 0s - loss: 1.8145 - accuracy: 0.2477
Epoch 00001: val_loss improved from inf to 1.52547, saving model to inception_model.h5
759/759 [==============================] - 38s 44ms/step - loss: 1.8142 - accuracy: 0.2477 - val_loss: 1.5255 - val_accuracy: 0.3195
Epoch 2/25
759/759 [==============================] - ETA: 0s - loss: 1.5810 - accuracy: 0.2790
Epoch 00002: val_loss improved from 1.52547 to 1.51854, saving model to inception_model.h5
759/759 [==============================] - 32s 42ms/step - loss: 1.5810 - accuracy: 0.2790 - val_loss: 1.5185 - val_accuracy: 0.3226
Epoch 3/25
759/759 [==============================] - ETA: 0s - loss: 1.5524 - accuracy: 0.3015
Epoch 00003: val_loss did not improve from 1.51854
759/759 [==============================] - 32s 42ms/step - loss: 1.5524 - accuracy: 0.3015 - val_loss: 1.6562 - val_accuracy: 0.2217
Epoch 4/25
759/759 [==============================] - ETA: 0s - loss: 1.5481 - accur